<a href="https://colab.research.google.com/github/visiont3lab/deep-learning-course/blob/main/colab/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Library

In [2]:
from torch import nn
from torch.utils.data import DataLoader
from torch import optim
import torch
from torch import nn
from torchsummary import summary
#!pip install torchsummary
import torch.nn.functional as F
from torch.utils.data import TensorDataset,Dataset
from torchvision import datasets
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import numpy as np
import plotly.graph_objects as go
# Loss function pytorch: https://neptune.ai/blog/pytorch-loss-functions

import pandas as pd
from datetime import datetime

## Neural Network 

## Dataset

In [6]:
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/visiont3lab/project-work-ifoa/main/data/dpc-covid19-ita-regioni-zone.csv")
df["data"] = [ datetime.strptime(d, "%Y-%m-%d %H:%M:%S") for d in df["data"]]
#df_f = df[df["data"]>datetime(2020,11,6)].copy()
df_f = df[df["zona"]!="unknown"]
inputs = ["data","denominazione_regione","zona","ricoverati_con_sintomi","terapia_intensiva",
        "totale_ospedalizzati","totale_positivi","isolamento_domiciliare",
        "deceduti","dimessi_guariti","nuovi_positivi","totale_casi","tamponi"]
df_f = df_f[inputs]
df_f = df_f.drop(columns=["data","denominazione_regione"])
#display(df_f)



#C_Y = np.array([dict_names[d] for d in df_Y],dtype=np.float) #.reshape(-1,1)
#print(f"X shape: {C_X.shape} , Y shape: {C_Y.shape}")

dict_names = {"bianca":0,"gialla": 1, "arancione": 2, "rossa": 3}
C_Y = df_f.pop("zona").tolist()
C_Y = np.array ( [ dict_names[e] for e in C_Y] , dtype=np.float32).reshape(-1,1)

C_X = df_f.values

print( C_X.shape )
print( C_Y.shape )

(2688, 10)
(2688, 1)


## Neural Network


In [44]:
class ClassificationNet(nn.Module):
    def __init__(self,num_inputs, num_classes):
        super(ClassificationNet,self).__init__()
        self.num_classes = num_classes
        self.fc1 = nn.Linear(num_inputs,200)
        self.fc2 = nn.Linear(200,100)
        self.fc3 = nn.Linear(100,50)
        self.fc4 = nn.Linear(50,self.num_classes)
    def forward(self,x):
        # torch.sigmoid, torch.tanh, torch.relu
        x = torch.tanh(self.fc1(x)) 
        x = torch.tanh(self.fc2(x))
        x = torch.tanh(self.fc3(x))
        x = torch.log_softmax(self.fc4(x),dim=-1) # sarebbe dim=1  print(self.fc3(x)) print(self.fc3(x).sum(dim=-1))
        return x

CN = ClassificationNet(num_inputs=10, num_classes=4)
CN

ClassificationNet(
  (fc1): Linear(in_features=10, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=50, bias=True)
  (fc4): Linear(in_features=50, out_features=4, bias=True)
)

## Preprocessing

In [37]:
# Training and Test Set
C_X_train, C_X_test, C_Y_train, C_Y_test = train_test_split(C_X,C_Y,test_size=0.3,shuffle=True,random_state=2)
print(f"X Train shape: {C_X_train.shape} , X Test shape: {C_X_test.shape}")

# Normalization
C_mean = np.mean(C_X)
C_std = np.std(C_X)
C_min = np.min(C_X)
C_max = np.max(C_X)

# Tensor Dataset Che converte i dati da numpy a Pytorch
class CustomTensorDataset(Dataset):
    def __init__(self, x,y,mean,std):
        x = (x - mean)/std           # Standard Scaler 
        #x = (x - min) / (max - min) # Min Max Scaler
        self.x = torch.from_numpy(x).type(torch.float32)
        self.y = torch.from_numpy(y).type(torch.LongTensor).reshape(-1)
    def __getitem__(self, index):
        x = self.x[index]
        y = self.y[index]
        return x, y
    def __len__(self):
        return self.x.shape[0]

# Dataset generator creation
C_train_ds = CustomTensorDataset(C_X_train,C_Y_train,C_mean,C_std)
C_test_ds = CustomTensorDataset(C_X_test,C_Y_test,C_mean,C_std)

X Train shape: (1881, 10) , X Test shape: (807, 10)


## Training

* numero di epoche
* learning rate
* batch size
* aggiornare la struttura della rete

In [38]:
# Validation: Metric Regression
def metrics_func_regression(target, output):
  # Comptue mean squaer error (Migliora quanto piu' ci avviciniamo a zero)
  mse = torch.sum((output - target) ** 2)
  return mse

# Validation: Metric cassification
def metrics_func_classification(target, output):
  # Compute number of correct prediction
  pred = output.argmax(dim=-1,keepdim=True)
  corrects =pred.eq(target.reshape(pred.shape)).sum().item()
  return -corrects # minus for coeherence with best result is the most negative one

# Training: Loss calculation and backward step
def loss_batch(loss_func,metric_func, xb,yb,yb_h, opt=None):
  # obtain loss
  loss = loss_func(yb_h, yb)
  # obtain permormance metric 
  metric_b = metric_func(yb,yb_h)
  if opt is not None:
    loss.backward()
    opt.step()
    opt.zero_grad()
  return loss.item(), metric_b

# Trainig: Function 1 epoch
def loss_epoch(model, loss_func,metric_func, dataset_dl, opt, device):
  loss = 0.0
  metric = 0.0
  len_data = len(dataset_dl.dataset)
  # Get batch data
  for xb,yb in dataset_dl:    
    # Send to cuda the data (batch size)
    xb = xb.to(device)
    yb = yb.to(device)
    # obtain model output 
    yb_h = model.forward(xb)
    # Loss and Metric Calculation
    loss_b, metric_b = loss_batch(loss_func,metric_func, xb,yb,yb_h,opt)
    loss += loss_b
    if metric_b is not None:
      metric+=metric_b 
  loss /=len_data
  metric /=len_data
  return loss, metric

# Training: Iterate on epochs
def train_val(epochs, model, loss_func, metric_func, opt, train_dl,test_dl,device, path2weigths="./weights.pt"):
  lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(opt, gamma=0.999) #  lr = lr * gamma ** last_epoch
  best_val_metric = 1000000
  for epoch in range(epochs):
    model.train()
    train_loss,train_metric = loss_epoch(model, loss_func, metric_func,train_dl, opt,device)
    lr_scheduler.step()
    model.eval()
    with torch.no_grad():
      val_loss, val_metric = loss_epoch(model, loss_func, metric_func, test_dl,opt=None,device=device)
      print("epoch: %d, train_loss: %.6f, val loss: %.6f,  train_metric: %.3f test_metric: %.3f lr: %.5f)" % (epoch,train_loss, val_loss,train_metric,val_metric,opt.param_groups[0]['lr']))
      if (val_metric <= best_val_metric):        
        # Save Models (It save last weights)
        torch.save(model.state_dict(),path2weigths)
        best_val_metric = val_metric


In [45]:
# Setup GPU Device
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda:0")

# Regression
C_model = ClassificationNet(num_inputs=10, num_classes=4).to(device)
C_loss_func = nn.NLLLoss(reduction="sum")  #nn.BCELoss  
C_opt = optim.Adam(C_model.parameters(),lr=0.001)
C_train_dl = DataLoader(C_train_ds,batch_size=124,shuffle=True)
C_test_dl = DataLoader(C_test_ds,batch_size=124,shuffle=True)

# Regression
train_val(3000,C_model,C_loss_func,metrics_func_classification,C_opt, C_train_dl,C_test_dl,device,path2weigths="./weights_classification.pt")


epoch: 0, train_loss: 1.304989, val loss: 1.260151,  train_metric: -0.345 test_metric: -0.347 lr: 0.00100)
epoch: 1, train_loss: 1.271598, val loss: 1.261257,  train_metric: -0.330 test_metric: -0.347 lr: 0.00100)
epoch: 2, train_loss: 1.269783, val loss: 1.252971,  train_metric: -0.367 test_metric: -0.373 lr: 0.00100)
epoch: 3, train_loss: 1.264140, val loss: 1.252161,  train_metric: -0.384 test_metric: -0.347 lr: 0.00100)
epoch: 4, train_loss: 1.263188, val loss: 1.247354,  train_metric: -0.383 test_metric: -0.421 lr: 0.00100)
epoch: 5, train_loss: 1.261747, val loss: 1.246840,  train_metric: -0.369 test_metric: -0.400 lr: 0.00099)
epoch: 6, train_loss: 1.257173, val loss: 1.241778,  train_metric: -0.367 test_metric: -0.414 lr: 0.00099)
epoch: 7, train_loss: 1.256002, val loss: 1.239756,  train_metric: -0.385 test_metric: -0.393 lr: 0.00099)
epoch: 8, train_loss: 1.253820, val loss: 1.242387,  train_metric: -0.366 test_metric: -0.343 lr: 0.00099)
epoch: 9, train_loss: 1.256188, val l

## Testing

In [46]:
# Load Classification
names = list({"bianca":0,"gialla": 1, "arancione": 2, "rossa": 3})
C_model = ClassificationNet(num_inputs=10,num_classes=4).to(device)
C_weights = torch.load("weights_classification.pt")
C_model.load_state_dict(C_weights)
C_model = C_model.to(device)

# Predict Classication
C_Xt = torch.from_numpy(C_X_test).type(torch.float32).unsqueeze(1)
C_Xt = (C_Xt - C_mean) / C_std
C_Y_hat = C_model.forward(C_Xt).argmax(dim=-1,keepdim=True).detach().numpy().reshape(-1)

# Visualize results
cm = confusion_matrix(C_Y_test,C_Y_hat)
names_pred = [ "Pred: " + n for n in names]
df = pd.DataFrame(cm, columns=names_pred, index=names)
print(df)

           Pred: bianca  Pred: gialla  Pred: arancione  Pred: rossa
bianca              242            20               15            3
gialla                9           257                4            0
arancione            15            10              177            2
rossa                 2             0                0           51


## Extra

In [ ]:

# df.head() # prime 5 righe
# df.tail() # ultime 5 righe
# df.to_csv("a.csv") # Salva come csv
# df.keys() # nomi delle colonne
# df.values.shape # dimensione
# df.values # dati come array 

# Nomi delle colonne
# Numero di righe
# selezionare solo la colonna con nome dimessi_guariti
# voglio i dati come array
# voglio i deceduti > 700
# dati del 12 marzo in poi

In [ ]:
x = np.array( [5,3,5,6,7] , dtype=np.float32)
x = x.reshape(1,-1)
x

array([[5., 3., 5., 6., 7.]], dtype=float32)

In [ ]:
x = torch.tensor( [5,3,5,6,7] , dtype=torch.float32)
#x = x.reshape(1,-1)
#x = x.unsqueeze(0)
x.shape

torch.Size([5, 1])

In [ ]:
import pandas as pd

d = {
    "c" : [5,6,7,8,9],
    "e" : [15,26,37,18,29],
    "t" : [0.5,0.26,3.7,1.8,29], 
}

df = pd.DataFrame(d)
df.values

array([[ 5.  , 15.  ,  0.5 ],
       [ 6.  , 26.  ,  0.26],
       [ 7.  , 37.  ,  3.7 ],
       [ 8.  , 18.  ,  1.8 ],
       [ 9.  , 29.  , 29.  ]])

In [ ]:
x = torch.from_numpy(df.values).type(torch.float32)
x

tensor([[ 5.0000, 15.0000,  0.5000],
        [ 6.0000, 26.0000,  0.2600],
        [ 7.0000, 37.0000,  3.7000],
        [ 8.0000, 18.0000,  1.8000],
        [ 9.0000, 29.0000, 29.0000]])